In [ ]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
df = TabularDataset("new_branded_data.csv")
df = df.dropna(subset=["choice"])
df = df.drop(['id', 'Unnamed: 0','flight_departure_datetime', 'purchase_datetime', 'ticket_id'], axis=1)
df = df.dropna()
df

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
label = 'choice'
train_data[label].describe()

In [ ]:
predictor = TabularPredictor(label=label, eval_metric="f1_weighted").fit(
    train_data,
    time_limit=3600,
    presets="best_quality",
    hyperparameters="extreme",
)

In [ ]:
leaderboard = predictor.leaderboard(test_data)
leaderboard

In [ ]:
best_model_name = predictor.model_best
best_model = predictor._trainer.load_model(best_model_name)

In [ ]:
import pandas as pd
df_to_predict = pd.read_csv("new_branded_baseline.csv")

columns_to_drop = ['id',
                   'Unnamed: 0',
          'ticket_id',
          'choice',
          'flight_departure_datetime',
          'purchase_datetime']
X_to_predict = df_to_predict.drop(columns_to_drop, axis=1)
X_to_predict = X_to_predict.dropna()

In [ ]:
predictions = predictor.predict(X_to_predict)
predictions

In [ ]:
df_to_predict['choice'] = predictions
df_to_predict[['id', 'choice']].to_csv('autogluon_predictions.csv', index=False)